# Aula 05 - Avaliação de Sistemas de Recomendação - Exercícios

In [2]:
import pandas as pd
import numpy as np

### Importar base de dados

In [4]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz


Saved under ml-20m-compact.tar (25).gz


x dataset/
x dataset/tags_sample.csv
x dataset/._.DS_Store
x dataset/.DS_Store
x dataset/movies_sample.csv
x dataset/._genome-tags.csv
x dataset/genome-tags.csv
x dataset/._ml-youtube.csv
x dataset/ml-youtube.csv
x dataset/._genome-scores.csv
x dataset/genome-scores.csv
x dataset/ratings_sample.csv


In [5]:
movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
df

,userId,movieId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


In [6]:
movies_genres = movies.drop('genres', axis=1).join(movies.genres.str.split('|', expand=True)
             .stack().reset_index(drop=True, level=1).rename('genre'))
movies_genres.head()

,movieId,title,genre
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Drama
1,31,Dangerous Minds (1995),Drama
2,37,Across the Sea of Time (1995),Documentary
2,37,Across the Sea of Time (1995),IMAX


In [7]:
movies_tags = pd.read_csv('./dataset/tags_sample.csv')
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,279,916,Gregory Peck,1329962459
1,279,916,need to own,1329962471
2,279,916,romantic comedy,1329962476
3,279,916,Rome,1329962490
4,279,916,royalty,1329962474


In [8]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}

df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)

movies_tags['userId'] = movies_tags['userId'].map(map_users)
movies_tags['movieId'] = movies_tags['movieId'].map(map_items)
movies_tags.dropna(inplace=True)
movies_tags['movieId'] = movies_tags.movieId.astype(int)
movies_tags['userId'] = movies_tags.userId.astype(int)

movies_genres['movieId'] = movies_genres['movieId'].map(map_items)
movies_genres.dropna(inplace=True)
movies_genres['movieId'] = movies_genres.movieId.astype(int)

map_title = {}
for _, row in df.iterrows():
    map_title[row.movieId] = row.title


In [9]:
movies_genres[['movieId', 'genre']].to_csv('items_genres.dat', index=False, sep='\t', header=False)
movies_tags[['movieId', 'tag']].to_csv('items_tags.dat', index=False, sep='\t', header=False)

### Divisão da base em treino e teste

In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)
train.to_csv('train.dat', index=False, header=False, sep='\t')
test.to_csv('test.dat', index=False, header=False, sep='\t')
train

,userId,movieId,rating,title
51276,2985,88,2.5,"Prince & Me, The (2004)"
125583,7302,222,5.0,"Earrings of Madame de..., The (Madame de...) (..."
150073,8729,82,2.0,Australia (2008)
186571,10853,17,3.0,Chasing Amy (1997)
38278,2230,27,3.0,Zero Effect (1998)
...,...,...,...,...
162091,9437,12,4.0,Star Wars: Episode V - The Empire Strikes Back...
175638,10230,16,2.0,Rush Hour 2 (2001)
95816,5556,41,4.5,Dangerous Beauty (1998)
100879,5846,8,2.0,"Karate Kid, The (1984)"


### Exemplos de utilização do CaseRecommender

In [13]:
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN

ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=10, as_similar_first=True).compute()
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_tags.dat', k_neighbors=10, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 5.077804 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 0.708069 sec
Eval:: MAE: 0.73347 RMSE: 0.964352 
[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.414645 sec
>> metadata:: 231 items and 1979 metadata (6274 interactions) | sparsity:: 98.63%
prediction_time:: 0.757888 sec
Eval:: MAE: 0.765116 RMSE: 1.006911 


***Exercício 01:*** Verifique o efeito no RSME ao aumentar o número de vizinhos do algoritmo ItemAttributeKNN com usando gêneros. Explique.

In [44]:
# TODO
k_neighbours = [10, 20, 50, 100, 200, 500]
for k in k_neighbours:
    model = ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=k, as_similar_first=True,)
    model.compute( verbose=False, verbose_evaluation=False)
    print(f"K = {k}, RSME = {model.evaluation_results['RMSE']}")
    

K = 10, RSME = 0.964352
K = 20, RSME = 0.969717
K = 50, RSME = 0.965269
K = 100, RSME = 0.945425
K = 200, RSME = 0.920997
K = 500, RSME = 0.916212


O aumento no número de vizinhos *K* geralmente leva a uma melhoria na precisão das previsões, como indicado pela redução do RMSE. Porém, para a escolha ideal de *K* deve-se balancear entre ter um número suficiente de vizinhos para capturar padrões gerais e não ser tão grande a ponto de haver perdas de informações específicas.

***Exercício 02:*** Verifique o efeito na métrica MAP@1 ao aumentar o número de fatores do algoritmo BPRMF. Explique.

In [18]:
# TODO
from caserec.recommenders.item_recommendation.bprmf import BprMF
n_factors = [1, 4, 7, 10, 15, 20, 25, 30]
for n in n_factors:
    model = BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=n)
    model.compute( verbose=False, verbose_evaluation=False)
    print(f"Factors = {n}, MAP@1 = {model.evaluation_results['MAP@1']}")

Factors = 1, MAP@1 = 0.383124
Factors = 4, MAP@1 = 0.359096
Factors = 7, MAP@1 = 0.347744
Factors = 10, MAP@1 = 0.36515
Factors = 15, MAP@1 = 0.370353
Factors = 20, MAP@1 = 0.37338
Factors = 25, MAP@1 = 0.397786
Factors = 30, MAP@1 = 0.408381


O aumento no número de fatores geralmente melhora a métrica MAP@1 (*Mean Average Precision at 1*) , o que indica que o modelo BPRMF está se tornando mais eficaz em recomendar o filme relevante no topo da lista. Isso ocorre porque mais fatores permitem uma representação mais detalhada e precisa das preferências dos usuários e das características dos filmes. Como visto acima, ao se ter um aumento significante no números de fatores, é possível notar um aumento na métrica MAP@1.

***Exercício 03:*** Implemente a métrica Average Precision para um único usuário. Utilize as recomendações geradas pelo algoritmo BPRMF (consulte notebook Aula04_Exemplo como base).

In [21]:
model = BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=3)
model.compute(verbose=False, verbose_evaluation=False)

recommendations = pd.read_csv('ir_bprmf.dat', sep='\t', names=['userId', 'movieId', 'score'])

In [22]:
# TODO

# Calculate the average precision for a user
def average_precision(user_id, recommendations, test): 
    # Get the recommendations for the specific user
    recs_user = recommendations.loc[(recommendations.userId == user_id), 'movieId'].tolist()
    # Get the relevant movies for the specific user
    relevant_movies = test.loc[(test.userId == user_id), 'movieId'].tolist()
    
    n_relevant_movies = 0
    cumulative_precision = 0.0
    
    # Iterate over the recommendations and calculate the precision
    for i, movie in enumerate(recs_user):
        if movie in relevant_movies:
            n_relevant_movies += 1
            # Precision at index i
            precision_at_i = n_relevant_movies / (i + 1)
            cumulative_precision += precision_at_i
    
    if n_relevant_movies == 0:
        return 0.0
    
    ap = cumulative_precision / n_relevant_movies
    return ap

# Calc the average precision for the first 10 user_ids
for u in range(1, 11):
    print(f"User: {u}, AP = {average_precision(u, recommendations, test):.4f}") 

User: 1, AP = 0.6250
User: 2, AP = 0.2500
User: 3, AP = 0.5000
User: 4, AP = 0.1000
User: 5, AP = 0.6429
User: 6, AP = 0.7500
User: 7, AP = 0.1250
User: 8, AP = 0.0000
User: 9, AP = 0.0000
User: 10, AP = 0.2083


***Exercício 04:*** Um importante aspecto que pode ser avaliado em Sistemas de Recomendação é a diversidade da lista de recomendações. A métrica Intra-List Similarity (ILS) aplica uma função de similaridade (e.g. Cosseno, Jaccard, Pearson, etc.) entre todos os pares de itens da lista de recomendação, usando seus metadados como gêneros, tags, etc. Mais detalhes sobre essa métrica podem ser encontrados em: https://grouplens.org/site-content/uploads/Improving-WWW-20051.pdf

***a)*** Implemente uma função que calcula a ILS de uma lista de recomendação para um único usuário. Utilize os gêneros de filmes.

In [25]:
# TODO

***b)*** Utilize a função que implementou no item (a) para calcular a ILS de todos os usuários da base de dados. Utilize as recomendações geradas pelo algoritmo BPR MF do CaseRecommender.

In [27]:
# TODO

***c)*** Aplique a função que implementou no item (b) para calcular a ILS das recomendações geradas pelos algoritmos ItemKNN e ItemAttributeKNN. Compare os resultados de todos os algoritmos, verificando inclusive o que acontece com a acurácia (precisão, MAP ou NDCG) quando a diversidade aumenta ou diminui.

In [29]:
# TODO